### ARES

In [3]:
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


df = pd.read_csv('document/clean_content_4.csv')


loader = DataFrameLoader(df, page_content_column="content")
docs = loader.load()

CHUNK_SIZE = 1024
CHUNK_OVERLAP = 0

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, add_start_index=True
)

all_splits = child_splitter.split_documents(docs)
print(f'len(all_splits): {len(all_splits)}')


len(all_splits): 19092


In [16]:
raw_df = pd.read_csv('document/clean_content_4.csv')
# raw_df['url'] = raw_df.index
raw_df2 = raw_df.set_index('url')
# raw_df.reset_index(inplace=True, index=raw_df['url'])
raw_df2.head()

,timestamp,content
url,,
https://www.ac.ntu.edu.tw/,1.702928e+09,Close login 登入 國立臺灣大學 農業化學系 忘記密碼? 國立臺灣大學 農業化學系...
https://www.aca.ntu.edu.tw/w/aca/,1.702928e+09,跳到主要內容 En 全站搜尋 search En 點擊時關閉搜尋 點擊時前往搜尋 close...
https://ga.ntu.edu.tw/main_ch/,1.702928e+09,國立臺灣大學總務處 國立臺灣大學總務處 連結各組 為順利達成112年度內各項付款及113年1...
https://ord.ntu.edu.tw/w/ordNTU/,1.702928e+09,跳到主要內容 請輸入關鍵字 - 送出的按鈕 計畫申請 人員管理 補助獎項 研究資源 研究倫理...
https://oia.ntu.edu.tw/,1.702928e+09,焦點 公告 一覽 Global NTU Oceania Asia Europe Africa...


In [5]:
document_df = pd.DataFrame({'Document': [doc.page_content for doc in all_splits]})
document_df.to_csv('ARES/document.tsv', index=False, sep='\t')

In [52]:
faq = pd.read_csv('document/FAQ_filter.csv')
faq.head()

,URL,Question,Answer
0,https://visitorcenter.ntu.edu.tw/News_Content_...,1.我是本校學生，請問有關 (1)領取/換發學生證（遺失補申請）、上課證；(2)申請成績單；...,以上問題，若你是大學生，請洽註冊組；若為研究生，請洽研教組。\n同時，歡迎多加利用自動申請機...
1,https://visitorcenter.ntu.edu.tw/News_Content_...,2.我想報名轉學考試，應該到哪間辦公室？,請洽課務組，相關位置請參考網頁「洽公須知」第一行政大樓2樓平面圖。\n\n課務組電話：02-...
2,https://visitorcenter.ntu.edu.tw/News_Content_...,3.我想申請獎學金、辦理就學貸款、學雜費減免、學生事病假單、學生保險、填寫失物招領？,請洽生活輔導組，相關位置請參考網頁「洽公須知」第一行政大樓1樓平面圖。\n\n生活輔導組電話...
3,https://visitorcenter.ntu.edu.tw/News_Content_...,5.我想要申請社團活動、張貼海報……等,發布單位：綜合業務組\n\n請洽課外活動組，相關位置請參考網頁「洽公須知」第一行政大樓1樓平...
4,https://visitorcenter.ntu.edu.tw/News_Content_...,6.我是本校畢業的校友，請問申辦校友證、校友捐贈、協尋畢業校友、參加校友活動?,請洽校友中心，位於本校總圖書館1F。\n\n校友中心電話：(02) 3366-3299


In [27]:
few_shot_examples = {'Query':faq['Question'], 'Document': [], 'Answer': faq['Answer'], 
                    'Contradictory_Answer': [], 'Context_Relevance_Label': [],
                    'Answer_Faithfulness_Label': [], 'Answer_Relevance_Label': []}

few_shot_examples['Document'] = [raw_df2['content'][url] for url in faq['URL']]

for url in faq['URL']:
    for idx, url2 in enumerate(faq['URL']):
        if url2!=url:
            few_shot_examples['Contradictory_Answer'].append(faq['Answer'][idx])
            break

few_shot_examples['Context_Relevance_Label'] = ['YES']*len(few_shot_examples['Document'])
few_shot_examples['Answer_Faithfulness_Label'] = ['YES']*len(few_shot_examples['Document'])
few_shot_examples['Answer_Relevance_Label'] = ['YES']*len(few_shot_examples['Document'])
few_shot_prompt_df = pd.DataFrame(few_shot_examples)
few_shot_prompt_df = few_shot_prompt_df.dropna()


few_shot_prompt_df.to_csv('ARES/few_shot_prompt.tsv', index=False, sep='\t')

In [53]:
few_shot_examples = {'Query':faq['Question'], 'Document': [], 'Answer': faq['Answer'], 
                    'Context_Relevance_Label': [],
                    'Answer_Faithfulness_Label': [], 'Answer_Relevance_Label': []}

few_shot_examples['Document'] = [raw_df2['content'][url] for url in faq['URL']]

# for url in faq['URL']:
#     for idx, url2 in enumerate(faq['URL']):
#         if url2!=url:
#             few_shot_examples['Contradictory_Answer'].append(faq['Answer'][idx])
#             break

few_shot_examples['Context_Relevance_Label'] = ['1']*len(few_shot_examples['Document'])
few_shot_examples['Answer_Faithfulness_Label'] = ['1']*len(few_shot_examples['Document'])
few_shot_examples['Answer_Relevance_Label'] = ['1']*len(few_shot_examples['Document'])
few_shot_prompt_df = pd.DataFrame(few_shot_examples)
few_shot_prompt_df = few_shot_prompt_df.dropna()


few_shot_prompt_df.to_csv('ARES/evaluation_datasets.tsv', index=False, sep='\t')

### RAGAS

In [2]:
import pandas as pd

faq = pd.read_csv('document/testset.csv')
faq.head(1)

,Unnamed: 0,URL,Question,Answer
0,0,https://topic.tipo.gov.tw/copyright-tw/cp-415-...,如何聯絡智慧財產局著作權組？,您可以透過以下方式聯絡智慧財產局著作權組：\n- 電話：著作權諮詢專線電話\n- 電子郵件：...


In [3]:
# check ragas

cnt = 0

raw_df = pd.read_csv('document/clean_content_4.csv')
# raw_df['url'] = raw_df.index
raw_df2 = raw_df.set_index('url')
# raw_df.reset_index(inplace=True, index=raw_df['url'])
raw_df2.head()

for idx, doc in enumerate([raw_df2['content'][url] for url in faq['URL']]):
    if 0< len(str(doc)) < 20:
        print(str(doc), faq['URL'][idx])
        cnt+=1

print(f'{cnt}/{len(faq)} questions crawl fail in testset.')

0/997 questions crawl fail in testset.


In [59]:
# hyde = pd.read_csv('document/FAQ_HyDE.csv')
# hyde.head(1)

,URL,Question,Answer,ChatGPT,retreival doc,retreival url,HyDE
0,https://admissions.ntu.edu.tw/zh-hant/apply/faq/,學生身份別之間有什麽差異？我應該如何查詢我的身份別呢？,學生身份別一共分四種：國際生、僑生、港澳生及陸生。您可以透過境外學位生身份別測驗來查詢自己的...,學生身份別有哪些類別？我該如何確認自己的身份別呢？,['to pay tuition? 我應該去哪裡得到學費繳交訊息？ To access yo...,['http://www.ipcs.ntu.edu.tw/page/about/index....,學生身份有多種類別，主要分為全日制學生和非全日制學生。全日制學生是指在學校正式註冊並按照學校...


In [65]:
# ans = []
# for f in faq.iterrows():
#     for h in hyde.iterrows():
#         if f[1]['Question'] == h[1]['Question']:
#             ans.append(h[1]['HyDE'])
#             break
# len(ans)

499

In [66]:
# ragas_dict = {'question':faq['Question'], 'contexts':[raw_df2['content'][url] for url in faq['URL']], 'answer': ans, 'ground_truths': faq['Answer']}

# origin_chatgpt_ans_df = pd.DataFrame(ragas_dict)
# origin_chatgpt_ans_df.head()

,question,contexts,answer,ground_truths
0,1.我是本校學生，請問有關 (1)領取/換發學生證（遺失補申請）、上課證；(2)申請成績單；...,跳到主要內容區塊 臺大訪客中心 洽公常見問答 mobile botton search 搜尋...,你好！對於領取/換發學生證（遺失補申請）、上課證，申請成績單，申請在學證明，休學、復學的申請...,以上問題，若你是大學生，請洽註冊組；若為研究生，請洽研教組。\n同時，歡迎多加利用自動申請機...
1,2.我想報名轉學考試，應該到哪間辦公室？,跳到主要內容區塊 臺大訪客中心 洽公常見問答 mobile botton search 搜尋...,如果你想報名轉學考試，你應該前往學校的教務處辦公室。教務處負責處理學生的轉學事務，包括轉學考...,請洽課務組，相關位置請參考網頁「洽公須知」第一行政大樓2樓平面圖。\n\n課務組電話：02-...
2,3.我想申請獎學金、辦理就學貸款、學雜費減免、學生事病假單、學生保險、填寫失物招領？,跳到主要內容區塊 臺大訪客中心 洽公常見問答 mobile botton search 搜尋...,您可以找學校的學生事務處或學生資助中心辦理獎學金申請、就學貸款、學雜費減免等相關事宜。他們通...,請洽生活輔導組，相關位置請參考網頁「洽公須知」第一行政大樓1樓平面圖。\n\n生活輔導組電話...
3,5.我想要申請社團活動、張貼海報……等,跳到主要內容區塊 臺大訪客中心 洽公常見問答 mobile botton search 搜尋...,如果您想申請社團活動或張貼海報等事宜，您可以找學校的學生事務處或學生活動中心。這些單位通常負...,發布單位：綜合業務組\n\n請洽課外活動組，相關位置請參考網頁「洽公須知」第一行政大樓1樓平...
4,6.我是本校畢業的校友，請問申辦校友證、校友捐贈、協尋畢業校友、參加校友活動?,跳到主要內容區塊 臺大訪客中心 洽公常見問答 mobile botton search 搜尋...,作為本校的校友，您可以到校友事務處辦理校友證、校友捐贈、協尋畢業校友以及參加校友活動。\n\...,請洽校友中心，位於本校總圖書館1F。\n\n校友中心電話：(02) 3366-3299


In [67]:
# origin_chatgpt_ans_df.to_csv('evaluation/chatgpt.csv', index=False)